## Imports 

In [1]:
# Needs iqm-benchmarks from the github repo to access all the mGST functions: https://github.com/iqm-finland/iqm-benchmarks
from mGST.low_level_jit import dK, objf, ddM, dK_dMdM
from mGST.optimization import tangent_proj
from mGST import additional_fns, algorithm, compatibility
from iqm.benchmarks.compressive_gst.compressive_gst import GSTConfiguration, CompressiveGST
from iqm.benchmarks.compressive_gst.gst_analysis import dataset_counts_to_mgst_format

from mGST.qiskit_interface import qiskit_gate_to_operator

import numpy as np

backend = "iqmfakeapollo"

ImportError: cannot import name 'CompressiveGST' from 'iqm.benchmarks.compressive_gst.compressive_gst' (/Users/emiliano.godinez/Documents/phd/iqm-benchmarks/src/iqm/benchmarks/compressive_gst/compressive_gst.py)

## Running the circuits on a simulator

In [2]:
Q2_GST = GSTConfiguration(
    qubit_layouts=[[0, 1]],
    gate_set="2QXYCZ",
    num_circuits=800,
    shots=1000,
    rank=4,
)

# Example configurations for 1 and 3 qubits:
# Q1_GST = GSTConfiguration(
#     qubit_layouts=[[0]],
#     gate_set="1QXYI",
#     num_circuits=100,
#     shots=1000,
#     rank=4,
# )

# Q3_GST = GSTConfiguration(
#     qubit_layouts=[[0,1,3]],
#     gate_set="3QXYCZ",
#     num_circuits=1000,
#     shots=1000,
#     rank=4,
# )

In [4]:
benchmark = CompressiveGST(backend, Q2_GST)
result = benchmark.run()

2025-01-07 11:41:23,842 - iqm.benchmarks.logging_config - INFO - Now generating 800 random GST circuits...
2025-01-07 11:41:24,492 - iqm.benchmarks.logging_config - INFO - Will transpile all 800 circuits according to fixed physical layout
2025-01-07 11:41:24,493 - iqm.benchmarks.logging_config - INFO - Transpiling for backend IQMFakeApolloBackend with optimization level 0, sabre routing method all circuits
2025-01-07 11:41:27,597 - iqm.benchmarks.logging_config - INFO - Submitting batch with 800 circuits corresponding to qubits [0, 1]
2025-01-07 11:41:27,647 - iqm.benchmarks.logging_config - INFO - Now executing the corresponding circuit batch
2025-01-07 11:41:27,795 - iqm.benchmarks.logging_config - INFO - Retrieving all counts


## Renaming the parameters for the convention used in the derivatives

In [5]:
qubit_layout = [0, 1]
dataset = result.dataset
y = dataset_counts_to_mgst_format(dataset, qubit_layout)
J = dataset.attrs["J"]
l = dataset.attrs["seq_len_list"][-1]
d = dataset.attrs["num_gates"]
pdim = dataset.attrs["pdim"]
r = pdim ** 2
n_povm = dataset.attrs["num_povm"]
bsize = dataset.attrs["batch_size"]
meas_samples = dataset.attrs["shots"]

rK = 4 # Setting the Kraus rank

# Setting some additional matrix shape parameters for the first and second derivatives
n = rK * pdim
nt = rK * r

## Initialization

In [6]:
## Preparing an initialization (random gate set or target gate set)

target_init = True

if target_init:
    K_target = qiskit_gate_to_operator(dataset.attrs["gate_set"])
    X_target = np.einsum("ijkl,ijnm -> iknlm", K_target, K_target.conj()).reshape(
        (dataset.attrs["num_gates"], dataset.attrs["pdim"] ** 2, dataset.attrs["pdim"] ** 2)
    )  # tensor of superoperators
    
    rho = (
        np.kron(additional_fns.basis(dataset.attrs["pdim"], 0).T.conj(), additional_fns.basis(dataset.attrs["pdim"], 0))
        .reshape(-1)
        .astype(np.complex128)
    )
    
    # Computational basis measurement:
    E = np.array(
        [
            np.kron(
                additional_fns.basis(dataset.attrs["pdim"], i).T.conj(), additional_fns.basis(dataset.attrs["pdim"], i)
            ).reshape(-1)
            for i in range(dataset.attrs["pdim"])
        ]
    ).astype(np.complex128)
    
    
    K = additional_fns.perturbed_target_init(X_target, dataset.attrs["rank"])
    X = np.einsum("ijkl,ijnm -> iknlm", K, K.conj()).reshape((d, r, r))
else:
    K, X, E, rho = random_gs(d, r, rK, n_povm)

## Gradient and Hessian

The function dK computes the Wirtinger derivative $\frac{\partial \mathcal L}{\partial K}$.
Here $\mathcal L$ is the cost function "objf".

In [7]:
# Euclidean Gradient
dK_ = dK(X, K, E, rho, J, y, d, r, rK)

The following code computes the Wirtinger derivatives "Fyconjy" $= \frac{\partial^2 \mathcal L}{\partial K \partial K^*}$ and "Fyy" $= \frac{\partial^2 \mathcal L}{\partial K \partial K}$. \
Here $\mathcal L$ is the cost function "objf".

In [8]:
# Euclidean Hessian (can take a while to compute depending on rK)
# compute individual second derivative terms
dK_, dM10, dM11 = dK_dMdM(X, K, E, rho, J, y, d, r, rK)
dd, dconjd = ddM(X, K, E, rho, J, y, d, r, rK)

# Assemple terms
Fyconjy = dM11.reshape(d, nt, d, nt) + np.einsum("ijklmnop->ikmojlnp", dconjd).reshape((d, nt, d, nt)) # Mixed derivate by K and K.conj()
Fyy = dM10.reshape(d, nt, d, nt) + np.einsum("ijklmnop->ikmojlnp", dd).reshape((d, nt, d, nt)) # Second derivate by K

In [9]:
print(K.shape)
print(Fyy.shape, d, nt)
# The second derivative is ordered with d = "nubmer of gates" and nt = pdim*pdim*rK = "Product of all Kraus tensor dimenstions per gate"
# So for instance the second derivative just by gate 0 - parameters is stored in Fyy[0,:,0,:], while a mixed derivative by gate 0 and gate 1 - parameters is in Fyy[0,:,1,:] and Fyy[1,:,0,:]

(5, 4, 4, 4)
(5, 64, 5, 64) 5 64
